# 数据准备

首先从[网站](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)上下载数据集，数据源分为SQL和CSV格式的，这里使用的事CSV格式的数据。
数据一共分成三部分：
1. 书：ISBN编号、书名、作者、出版年限、出版社、图片尺寸网址等信息
2. 用户：用户ID、位置信息、年龄
3. 评分：用户ID、ISBN编号、评分(0-10)

其中有278858名用户对271379本图书的1048575条评分。

In [1]:
# 引入相关模块
import pandas as pd
import numpy as np
import codecs
import zipfile

%matplotlib inline

In [2]:
def loadData():
    # 解压压缩包
    user_book_rating = zipfile.ZipFile('./dataset/BX-Dump.zip')
    user_book_rating.extractall("./dataset")
    user_book_rating.close()
    #读取csv文件
    #因为文件中有引号作为特殊字符，无法用pandas加载
    #所以此处手动加载数据
    
    # 加载图书数据，获取ISBN和其他信息
    books = {}
    with codecs.open('./dataset/BX-Books.csv', 'r', 'utf-8') as book:
        for line in book:
            features = list(map(lambda x: x.strip('"'), line.split(';')))
            books[features[0]] = '--'.join(features[1:])
                
    #  加载用户信息，获取用户编号和其他信息
    users = {}
    with codecs.open('./dataset/BX-Users.csv', 'r', 'utf-8') as user:
        for line in user:
            features = list(map(lambda x: x.strip('"'), line.split(';')))
            users[features[0]] = "--".join(features[1:])
            
    # 获取用户对书籍的评分，使用嵌套字典存储
    ratings = {}
    with codecs.open('./dataset/BX-Book-Ratings.csv', 'r', 'utf-8') as rating:
        for line in rating:
            features = list(map(lambda x: x.strip('"'), line.split(';')))
            if features[0] in ratings:
                current_ratings = ratings[features[0]]
            else:
                current_ratings = {}
            current_ratings[features[1]] = int(features[2].strip().strip('"'))
            ratings[features[0]] = current_ratings
#     books = pd.read_csv('./dataset/BX-Books.csv', sep=';')
#     user = pd.read_csv('./dataset/BX-Users.csv', sep=';')
#     ratings = pd.read_csv('./dataset/BX-Books-Ratings.csv', sep=';')
    
    return books, users, ratings

In [3]:
 books, users, ratings = loadData()

In [4]:
ratings["276725"]

{'034545104X': 0}

# 定义相似度验证方法
## 曼哈顿距离
曼哈顿距离是最简单的距离计算方式，一个点$(x, y)$由两个维度表示，两个点的曼哈顿距离就是在求得$x$轴之差的绝对值加上$y$轴只差的绝对值：
$$d= |x_1 - x_2| + |y_1 - y_2|$$
## 欧几里得距离
欧几里得距离可以计算任意维度中两个点之间的距离：
$$d = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$$
## 闵可夫斯基距离
曼哈顿距离和欧几里得距离可以统筹成一个距离公式，就是闵可夫斯基距离：
$$d(x,y) = (\sum_{k=1}^{n}|x_k - y_k|^r)^{\frac{1}{r}} $$
* r = 1 时是曼哈顿距离
* r = 2 时是欧几里得距离

## 皮尔逊相关系数
皮尔逊相关系数用于衡量两个变量之间的相关性，它的值在$-1$到$1$之间， $-1$表示完全相反， $1$表示完全相关：
$$r = \frac{\sum_{i=1}^{n}(x_i - \overline{x})(y_i - \overline{y})}{\sqrt{\sum_{i=1}^{n}(x_i-\overline{x})^2}\sqrt{\sum_{i=1}^{n} (y - \overline{y})^2}} \approx \frac{\sum_{i=1}^{n}x_iy_i-\frac{\sum_{i=1}^{n}  x_i \sum_{i=1}^{n}  y_i}{n}}{\sqrt{\sum_{i=1}^{n} x_i^2 - \frac{(\sum_{i=1}^{n} x_i)^2}{n}}\sqrt{\sum_{i=1}^{n} y_i^2 - \frac{(\sum_{i=1}^{n} y_i)^2}{n}}}$$

## 余弦相似度
余弦相似度可以解决数据的稀疏性问题，其在计算过程中会略过$0$值：
$$\cos{x, y} = \frac{x \cdot y}{||x||||y||}$$

In [5]:
# 曼哈顿距离
def manhattan(rating1, rating2):
    distance = 0
    for key in rating1:
        if key in rating2:
            distance += abs(rating1[key] - rating2[key])
    return distance

# 欧几里得距离
def euclid(rating1, rating2):
    distance = 0
    for key in rating1:
        if key in rating2:
            distance += np.square(rating1[key] - rating2[key])
    return np.sqrt(distance)

# 皮尔逊相关系数
def pearson(rating1, rating2):
    sum_xy = 0
    sum_x = 0
    sum_y = 0
    sum_x2 = 0
    sum_y2 = 0
    n = 0
    for key in rating1:
        if key in rating2:
            n += 1
            x = rating1[key]
            y = rating2[key]
            sum_xy += x + y
            sum_x += x
            sum_y += y
            sum_x2 += np.square(x)
            sum_y2 += np.square(y)
    if n == 0:
        return 0
    denominator = np.sqrt(sum_x2 - np.square(sum_x) / n) * np.sqrt(sum_y2 - np.square(sum_y) / n)
    if denominator == 0:
        return 0
    else:
        return (sum_xy - sum_x * sum_y / n)/denominator

# 余弦相似度
def cossim(rating1, rating2):
    sumxy = 0
    sumx2 = 0
    sumy2 = 0
    for key in rating1:
        if key in rating2:
            sumxy += rating1[key] * rating2[key]
            sumx2 += np.square(rating1[key])
            sumy2 += np.square(rating2[key])
    denominator = np.sqrt(sumx2*sumy2)
    if denominator == 0 :
        return 0
    else:
        return sumxy / denominator

In [6]:
# 计算与用户相邻的其他用户
def computerNearestNeighbor(user, fn=pearson):
    distances = []
    for instance in ratings:
        if instance != user:
            distance = fn(ratings[instance], ratings[user])
            distances.append((instance, distance))
    distances.sort(key=lambda x:x[1], reverse=True)
    return distances

# 开始推荐

In [7]:
def recommend(user, fn=pearson, k=1, n=5):
    """
    user 推荐的用户
    fn 相似度计算方法
    k  k临近算法中的值
    n 推荐几个
    """
    # 推荐列表
    recommendations = {}
    nearset = computerNearestNeighbor(user, fn)
    
    #获取用户评价过的商品
    userRatings = ratings[user]
    
    totalDistance = 0
    for i in range(k):
        totalDistance += nearset[i][1]
    for i in range(k):
        # 计算k个用户的权重
        weight = nearset[i][1] / totalDistance
        # 用户名称
        userId = nearset[i][0]
        # 获取该用户的评分
        neighborRatings = ratings[userId]
        # 获取推荐用户没有评价过的商品
        for item in neighborRatings:
            if item  not in userRatings:
                if item  not in recommendations:
                    recommendations[item] = (neighborRatings[item] * weight)
                else:
                    recommendations[item] += neighborRatings[item]*weight
    recommendations = list(recommendations.items())
    recommendations = [(books[k] if k in books else k, v) for (k, v) in recommendations]
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:n]

In [9]:
recommend('250764')

[('The Saving Graces: A Novel--Patricia Gaffney--2000--HarperTorch--http://images.amazon.com/images/P/0061097101.01.THUMBZZZ.jpg--http://images.amazon.com/images/P/0061097101.01.MZZZZZZZ.jpg--http://images.amazon.com/images/P/0061097101.01.LZZZZZZZ.jpg"\n',
  10.0),
 ('Tuesdays with Morrie: An Old Man, a Young Man, and Life\'s Greatest Lesson--MITCH ALBOM--1997--Doubleday--http://images.amazon.com/images/P/0385484518.01.THUMBZZZ.jpg--http://images.amazon.com/images/P/0385484518.01.MZZZZZZZ.jpg--http://images.amazon.com/images/P/0385484518.01.LZZZZZZZ.jpg"\n',
  10.0),
 ('Harry Potter and the Order of the Phoenix (Book 5)--J. K. Rowling--2003--Scholastic--http://images.amazon.com/images/P/043935806X.01.THUMBZZZ.jpg--http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg--http://images.amazon.com/images/P/043935806X.01.LZZZZZZZ.jpg"\n',
  10.0),
 ('A Time to Kill--JOHN GRISHAM--1992--Dell--http://images.amazon.com/images/P/0440211727.01.THUMBZZZ.jpg--http://images.amazon.com/images